In [1]:
from fastai.text import *
import os

In [2]:
path = os.getcwd()

In [3]:
def random_sample(file, frac):
    df = pd.read_csv(file)
    print(df.isna().sum())
    df_percent = df.sample(frac=frac)
    df_percent.columns =["text", "label"]
    df_percent = df_percent.iloc[:, [1,0]]
    df_percent.to_csv('subset.csv',index = False )
random_sample('preprocessed.csv', 0.01)

Tweet        1
Sentiment    0
dtype: int64


In [4]:
data_lm = TextLMDataBunch.from_csv(path, 'subset.csv')

In [5]:
data_cl = TextClasDataBunch.from_csv(path, 'subset.csv')

In [6]:
data_lm.save()
data_cl.save()

In [7]:
data_lm = TextLMDataBunch.load(path)
data_cl = TextClasDataBunch.load(path, bs=32)

In [10]:
learn = language_model_learner(data_lm, pretrained_model = URLs.WT103_1, 
                               drop_mult=0.4)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,5.627489,4.900580,0.177181


In [11]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy
1,4.733771,4.479630,0.219541


In [12]:
learn.freeze()
learn.fit_one_cycle(1,1e-2)

epoch,train_loss,valid_loss,accuracy
1,4.539777,4.354676,0.230323


In [13]:
learn.unfreeze()
learn.fit_one_cycle(1, 5e-3)

epoch,train_loss,valid_loss,accuracy
1,4.404706,4.273189,0.236413


In [14]:
learn.freeze()
learn.fit_one_cycle(1,1e-2)

epoch,train_loss,valid_loss,accuracy
1,4.205983,4.244975,0.237821


In [19]:
learn.predict("this is not")

'this is not up'

In [20]:
learn.save_encoder('ft_en')

In [21]:
learn = text_classifier_learner(data_cl, drop_mult=0.5)
learn.load_encoder('ft_en')

In [22]:
learn.fit_one_cycle(1,1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.611367,0.610949,0.696088


In [23]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy
1,0.585848,0.549218,0.732394


In [24]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy
1,0.547829,0.518923,0.747731


In [25]:
learn.freeze()
learn.fit_one_cycle(1,1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.554471,0.513900,0.746479


In [26]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy
1,0.526458,0.516766,0.745227


In [39]:
learn.predict("This tutorial is awesome")

(Category Positive, tensor(1), tensor([0.1827, 0.8173]))

In [42]:
learn.predict("I do not know why people use tensorflow instead of pytorch")

(Category Negative, tensor(0), tensor([0.6201, 0.3799]))